## Starting the Project

For this assignment, you can find the `smart cab` folder containing the necessary project files on the [Machine Learning projects GitHub](https://github.com/udacity/machine-learning), under the `projects` folder. You may download all of the files for projects we'll use in this Nanodegree program directly from this repo. Please make sure that you use the most recent version of project files when completing a project!

This project contains two directories:

- `/images/`: This folder contains various images of cars to be used in the graphical user interface. You will not need to modify or create any files in this directory.
- `/smartcab/`: This folder contains the Python scripts that create the environment, graphical user interface, the simulation, and the agents. You will not need to modify or create any files in this directory except for `agent.py`.

In `/smartcab/` are the following four files:
- **Modify:**
  - `agent.py`: This is the main Python file where you will be performing your work on the project.
- **Do not modify:**
  - `environment.py`: This Python file will create the **smartcab** environment.
  - `planner.py`: This Python file creates a high-level planner for the agent to follow towards a set goal.
  - `simulation.py`: This Python file creates the simulation and graphical user interface. 

### Environment
The **smartcab** operates in an ideal, grid-like city (similar to New York City), with roads going in the North-South and East-West directions. Other vehicles will certainly be present on the road, but there will be no pedestrians to be concerned with. At each intersection there is a traffic light that either allows traffic in the North-South direction or the East-West direction. U.S. Right-of-Way rules apply: 
- On a green light, a left turn is permitted if there is no oncoming traffic making a right turn or coming straight through the intersection.
-On a red light, a right turn is permitted if no oncoming traffic is approaching from your left through the intersection.

### Inputs and Outputs
The **smartcab** has only an egocentric view of the intersection it is at: It can determine the state of the traffic light for its direction of movement, and whether there is a vehicle at the intersection for each of the oncoming directions. For each action, the **smartcab** may either idle at the intersection, or drive to the next intersection to the left, right, or ahead of it. Finally, each trip has a time to reach the destination which decreases for each action taken (the passengers want to get there quickly).  If the allotted time becomes zero before reaching the destination, the trip has failed.

### Rewards and Goal
The **smartcab** receives a reward for each successfully completed trip, and also receives a smaller reward for each action it executes successfully that obeys traffic rules. The **smartcab** receives a small penalty for any incorrect action, and a larger penalty for any action that violates traffic rules or causes an accident with another vehicle. Based on the rewards and penalties the **smartcab** receives, the self-driving agent implementation should learn an optimal policy for driving on the city roads while obeying traffic rules, avoiding accidents, and reaching passengers? destinations in the allotted time.

## Tasks

### Project Report
You will be required to submit a project report along with your modified agent code as part of your submission. As you complete the tasks below, include thorough, detailed answers to each question *provided in italics*.

### Implement a Basic Driving Agent

To begin, your only task is to get the **smartcab** to move around in the environment. At this point, you will not be concerned with any sort of optimal driving policy. Note that the driving agent is given the following information at each intersection:
- The next waypoint location relative to its current location and heading.
- The state of the traffic light at the intersection and the presence of oncoming vehicles from other directions.
- The current time left from the allotted deadline.

To complete this task, simply have your driving agent choose a random action from the set of possible actions (`None`, `'forward'`, `'left'`, `'right'`) at each intersection, disregarding the input information above. Set the simulation deadline enforcement, `enforce_deadline` to `False` and observe how it performs.

***QUESTION:*** _Observe what you see with the agent's behavior as it takes random actions. Does the **smartcab** eventually make it to the destination? Are there any other interesting observations to note?_

**Answer**
The smartcab chooses randomly in actions and seems to perform a random walk in position as well as in rewards. The smartcab does make it to the destination after very many iterations. 

It is interesting to note, that the other agents seem to be moving around rather random as well. Furthermore, the reported state of the agent is always set to 'None'. Therefore you can see the state is not yet implemented/used in the agent.

### Inform the Driving Agent

Now that your driving agent is capable of moving around in the environment, your next task is to identify a set of states that are appropriate for modeling the **smartcab** and environment. The main source of state variables are the current inputs at the intersection, but not all may require representation. You may choose to explicitly define states, or use some combination of inputs as an implicit state. At each time step, process the inputs and update the agent's current state using the `self.state` variable. Continue with the simulation deadline enforcement `enforce_deadline` being set to `False`, and observe how your driving agent now reports the change in state as the simulation progresses.

***QUESTION:*** _What states have you identified that are appropriate for modeling the **smartcab** and environment? Why do you believe each of these states to be appropriate for this problem?_

***OPTIONAL:*** _How many states in total exist for the **smartcab** in this environment? Does this number seem reasonable given that the goal of Q-Learning is to learn and make informed decisions about each state? Why or why not?_

**Answer**

I believe the inputs as well as the self.next_waypoint values are important to determine the exact state the cab is in. Inputs are necessary since this determines whether a move is allowed by traffic rules or not. The next_waypoint is necessary to advise the cab what a good move would be, which is to say what direction would yield a reward, i.e. bring the cab closer to the end point. One could consider also using the deadline for the state, and how close one is to the destination. With a little work this could be used without exploding the possible state number (smart binning: closeness = near for distance < 4 points, etc, therefore binning into 2 bins). Then the cab could learn something like: If I now cross the road even if there is a red light I can still make it and get my reward. But for now I consider this too much, and rather a possibility for future add-ons. (This would still increase state-space by a factor 4)


To answer the optional question let's take a look at: inputs = {'light': 'red', 'oncoming': None, 'right': None, 'left': None}, action = left, reward = -1.0
next_waypoint: right.
The total number of states is just: 2 x 2 x 2 x 2 x 3 (green/red, None/yes, None/yes, None/yes, forward/left/right)
= 48 states in total. This seems to be a very reasonable number of states, that should be manageable to master for the Learning Agent.
For every state there are 4 possible actions to be taken. Therefore there are 192 values for the utility.

### Implement a Q-Learning Driving Agent

With your driving agent being capable of interpreting the input information and having a mapping of environmental states, your next task is to implement the Q-Learning algorithm for your driving agent to choose the *best* action at each time step, based on the Q-values for the current state and action. Each action taken by the **smartcab** will produce a reward which depends on the state of the environment. The Q-Learning driving agent will need to consider these rewards when updating the Q-values. Once implemented, set the simulation deadline enforcement `enforce_deadline` to `True`. Run the simulation and observe how the **smartcab** moves about the environment in each trial.

The formulas for updating Q-values can be found in [this](https://classroom.udacity.com/nanodegrees/nd009/parts/0091345409/modules/e64f9a65-fdb5-4e60-81a9-72813beebb7e/lessons/5446820041/concepts/6348990570923) video.

***QUESTION:*** _What changes do you notice in the agent's behavior when compared to the basic driving agent when random actions were always taken? Why is this behavior occurring?_

**Answer**

The agent very quickly learns to follow the next_waypoints. Therefore he quickly learns to reach the final destinations. Decisions are now rather independent from traffic light, therefore the agent basically learns to just follow the next_waypoints. Also, the agent rarely ever commits actions that will yield a penalty, after a few runs.

Before, the agent was just choosing an action randomly, not learning anything and not ever reaching the final destination.

The behaviour that is seen now occurs due to the memory effect that the utility has: Once in the same state as before, the agent will know that last time an action did not yield good results or that an action did lead to the wanted reward. Rewards and penalties lead to this effect, as a positive reward leads to a positive utility and a higher value of utility is preferred in the policy.

### Improve the Q-Learning Driving Agent

Your final task for this project is to enhance your driving agent so that, after sufficient training, the **smartcab** is able to reach the destination within the allotted time safely and efficiently. Parameters in the Q-Learning algorithm, such as the learning rate (`alpha`), the discount factor (`gamma`) and the exploration rate (`epsilon`) all contribute to the driving agent?s ability to learn the best action for each state. To improve on the success of your **smartcab**:
- Set the number of trials, `n_trials`, in the simulation to 100.
- Run the simulation with the deadline enforcement `enforce_deadline` set to `True` (you will need to reduce the update delay `update_delay` and set the `display` to `False`).
- Observe the driving agent?s learning and **smartcab?s** success rate, particularly during the later trials.
- Adjust one or several of the above parameters and iterate this process.

This task is complete once you have arrived at what you determine is the best combination of parameters required for your driving agent to learn successfully. 

***QUESTION:*** _Report the different values for the parameters tuned in your basic implementation of Q-Learning. For which set of parameters does the agent perform best? How well does the final driving agent perform?_

***QUESTION:*** _Does your agent get close to finding an optimal policy, i.e. reach the destination in the minimum possible time, and not incur any penalties? How would you describe an optimal policy for this problem?_

**Answer**
My initial parameters before were: alpha=0.1, gamma=0.1, epsilon=0.1. With these parameters the agent already quickly learned its way around and after the first run already reached every destination. I tried to make epsilon decay and start at a higher value, though that did not seem to change much. Therefore I would prefer to have a rather constant epsilon during training, and when you would actually want to use the trained smartcab, one should deactivate epsilon, as the optimal policy should be learned by then. Then I tried to make the agent learn what actions will result in penalties quicker. But I guess to learn this the agent has to randomly visit the consequences of all possible bad actions. In order to make it quicker, I started at a higher alpha of 0.5 and let it decay exponentially with alpha -= alpha/200. This seems to speed up the learning a bit. A gamma parameter of 0.2 also seems to increase the learning a bit.

I therefore choose these parameters as my final parameters: alpha=0.5 (decays over time), gamma = 0.2, epsilon=0.1.

This final driving agent reaches every destination. The first run with a deadline of 30 is reached with a deadline of 18 still left. Starting at about run 4 the agent seems to no longer receive any penalties except for when the action was taken randomly due to epsilon.

Therefore I would argue, that this agent learns quickly (reaches destination in 1st run; learns to avoid penalties quickly) and brings passengers safe (learns quick toa void penalties) and reliably(always reaches destination in time) to their destination.


There are things that are not done optimally yet. The agent right now basically learns to follow the next_waypoint and waits at every traffic light until it allows to travel to the next_waypoint. This is not efficient though. In this Manhatten street world, most often there are 2 optimal next_waypoints, not only one (only exception being when a straight line lies between destination and smartcab). Therefore at nearly all traffic light situations waiting is not the optimal action. But the cab waits anyway, as it does not know the final destination point but is only fed the next_waypoint. If the next_waypoint would allow for all optimal next_waypoints, or the cab was fed the final destination as well, it could learn this more optimal behaviour. I would argue, that adding the additional next_waypoint is the better solution, as the state-space stays at the same size and still allows for quick learning. Adding all the possible final destinations will make the state space bigger by a high factor (if I remember correctly 8 * 8 = 64), therefore making it very unlikely for the smartcab to visit all relevant states. A not yet learned final-destination will be a complete new state for the cab then, which is not really a desired situation.

The description of the actual agent in combination with the point just above would be my optimal policy, that is not yet completely fulfilled.

## Submitting the Project

### Evaluation
Your project will be reviewed by a Udacity reviewer against the **<a href="https://review.udacity.com/#!/rubrics/106/view" target="_blank">Train a Smartcab to Drive project rubric</a>**. Be sure to review this rubric thoroughly and self-evaluate your project before submission. All criteria found in the rubric must be *meeting specifications* for you to pass.

### Submission Files
When you are ready to submit your project, collect the following files and compress them into a single archive for upload. Alternatively, you may supply the following files on your GitHub Repo in a folder named `smartcab` for ease of access:
 - The `agent.py` Python file with all code implemented as required in the instructed tasks.
 - A **PDF** project report with the name **report.pdf** which answers all of the questions related to the tasks completed. This file *must* be present for your project to be evaluated.

Once you have collected these files and reviewed the project rubric, proceed to the project submission page.

In [5]:
import random
from smartcab.environment import Agent, Environment
from smartcab.planner import RoutePlanner
from smartcab.simulator import Simulator
import numpy as np
from collections import OrderedDict

random.seed(42)

class Utility():
    def __init__(self, valid_actions, valid_inputs, alpha=0.5, gamma=0.2, epsilon=0.1):
        self.Q_values = {}
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon
        self.valid_actions = valid_actions
        stateActionSpace = self.init_stateActionSpace(valid_actions, valid_inputs)
        for state_and_action_pair in stateActionSpace:
            self.Q_values[state_and_action_pair] = 0
    
    def update(self, old_state, action, reward, new_state):
        #Q(s,a) = (1-/alpha)Q(s,a) + /alpha ( R(s) + 
        #sum_over_all_possible_next_states_s' /gamma max(Q(s',a') , with regard of a'))
        oldStateKey = self.createStateActionKey(old_state, action)
        Q = self.Q_values[oldStateKey]
        Q_max = 0
        for action in self.valid_actions:
            newStateKey = self.createStateActionKey(new_state, action)
            Q_candidate = self.Q_values[newStateKey]
            if Q_candidate > Q_max:
                Q_max = Q_candidate
            
        Q = (1-self.alpha)*Q + self.alpha*(reward + self.gamma*Q_max)
        
        self.alpha -= self.alpha/200
        print "!!!! alpha = {}".format(self.alpha)
        self.Q_values[oldStateKey] = Q
    
    def policy(self, state):
        randomNumber = random.random()
        if randomNumber > (1-self.epsilon):
            print "!!! The following action is taken randomly !!!"
            return random.choice(self.valid_actions)
        else:
            Q_max = 0
            best_action = []
            for action in self.valid_actions:
                stateActionKey = self.createStateActionKey(state, action)
                Q = self.Q_values[stateActionKey]
                if Q > Q_max:
                    Q_max = Q
                    best_action = [action]
                if Q == Q_max:
                    best_action += [action]
                    
            if len(best_action)== 1:
                return best_action[0]
            else:
                return random.choice(best_action)
    
    def init_stateActionSpace(self, valid_actions, valid_inputs):
        possible_next_waypoints = valid_actions[1:]
        stateActionSpace = []
        state = {}
        for light in ['red','green']:
            for left in valid_inputs['left']:
                for right in valid_inputs['right']:
                    for oncoming in valid_inputs['oncoming']:
                        for next_waypoint in possible_next_waypoints:
                            for action in valid_actions:
                                state = {'light': light, 'next_waypoint': next_waypoint, 'right':right,
                                        'oncoming': oncoming, 'left': left}
                                stateActionSpace += [self.createStateActionKey(
                                    state, action)]
        return stateActionSpace
    
    def createStateActionKey(self, state, action):
        return ("{"+
                ("'light': {0!r}, 'next_waypoint': {1!r}, 'right': {2!r}, 'oncoming': {3!r},"
                "'left': {4!r}").format(state['light'], state['next_waypoint'], state['right'],
                                        state['oncoming'], state['left'])+
                "}" + 
                "action: {0!r}".format(action))
    
        
class LearningAgent(Agent):
    """An agent that learns to drive in the smartcab world."""

    def __init__(self, env):
        super(LearningAgent, self).__init__(env)  # sets self.env = env, state = None, 
                                                  # next_waypoint = None, and a default color
        self.color = 'red'  # override color
        self.planner = RoutePlanner(self.env, self)  # simple route planner to get next_waypoint
        
        # TODO: Initialize any additional variables here
        self.valid_actions = env.valid_actions
        self.valid_inputs = env.valid_inputs
        self.utility = Utility(self.valid_actions, self.valid_inputs)
    
    def reset(self, destination=None):
        self.planner.route_to(destination)
        # TODO: Prepare for a new trip; reset any variables here, if required

    def update(self, t):
        # Gather inputs
        self.next_waypoint = self.planner.next_waypoint()  # from route planner, also displayed by simulator
        inputs = self.env.sense(self)
        deadline = self.env.get_deadline(self)
        
        # TODO: Update state
        self.state = {'next_waypoint':str(self.next_waypoint)}
        self.state.update(inputs)
        old_state = self.state

        #Policy(s) = argmax(Q(s,a) , with regard to a)
        
        action = self.utility.policy(self.state)
    
        # Execute action and get reward
        reward = self.env.act(self, action)
        
        # TODO: Learn policy based on state, action, reward
        self.next_waypoint = self.planner.next_waypoint()  # from route planner, also displayed by simulator
        
        #get new state
        inputs = self.env.sense(self)
        self.state = {'next_waypoint':str(self.next_waypoint)}
        self.state.update(inputs)
        new_state = self.state
        
        #update utility
        self.utility.update(old_state, action, reward, new_state)
        
        print "next_waypoint: {}".format(self.next_waypoint)
        print "LearningAgent.update(): deadline = " \
        "{}, inputs = {}, action = {}, reward = {}".format(deadline, inputs, action, reward)  # [debug]
        
           

def run():
    """Run the agent for a finite number of trials."""

    # Set up environment and agent
    e = Environment()  # create environment (also adds some dummy traffic)
    a = e.create_agent(LearningAgent)  # create agent
    e.set_primary_agent(a, enforce_deadline=True)  # specify agent to track
    # NOTE: You can set enforce_deadline=False while debugging to allow longer trials

    # Now simulate it
    sim = Simulator(e, update_delay=0.5, display=False)  # create simulator 
    # (uses pygame when display=True, if available)
    # NOTE: To speed up simulation, reduce update_delay and/or set display=False

    sim.run(n_trials=100)  # run for a specified number of trials
    # NOTE: To quit midway, press Esc or close pygame window, or hit Ctrl+C on the command-line


if __name__ == '__main__':
    run()

Simulator.run(): Trial 0
Environment.reset(): Trial set up with start = (1, 6), destination = (4, 3), deadline = 30
RoutePlanner.route_to(): destination = (4, 3)
!!!! alpha = 0.4975
next_waypoint: right
LearningAgent.update(): deadline = 30, inputs = {'light': 'green', 'oncoming': None, 'right': None, 'left': None}, action = None, reward = 0.0
!!!! alpha = 0.4950125
next_waypoint: right
LearningAgent.update(): deadline = 29, inputs = {'light': 'green', 'oncoming': None, 'right': None, 'left': None}, action = None, reward = 0.0
!!! The following action is taken randomly !!!
!!!! alpha = 0.4925374375
next_waypoint: forward
LearningAgent.update(): deadline = 28, inputs = {'light': 'red', 'oncoming': None, 'right': None, 'left': None}, action = right, reward = 2.0
!!!! alpha = 0.490074750313
next_waypoint: forward
LearningAgent.update(): deadline = 27, inputs = {'light': 'green', 'oncoming': None, 'right': None, 'left': None}, action = None, reward = 0.0
!!!! alpha = 0.487624376561
next_wa